In [1]:
import sys, os
import numpy as np
import json
import pickle
from typing import Any, Union, Dict, List
import humpack
from humpack import tdict, tlist, tset
from humpack import secure as scr
from humpack import pack, unpack, pack_data, unpack_data

In [2]:
x = tdict()
x.a = 1
x.x = x
x.l = [tlist(), tset()]
x[100] = '100'
x[None] = 1.2
x.m = None
x.b = np.random.randn(3).tobytes()
x[b'\xaa'] = 'running'
t = x
t

t{'a':1, 'x':{...}, 'l':[t[], t{}], 100:'100', None:1.2, 'm':None, 'b':b'Y\x19\xe8F#\xdf\xdc?=\x85\xea#{\x81\xcd?\xca\\u\x012\xfa\xe2?', b'\xaa':'running'}

In [13]:
# x = scr.secure_pack(t)
x = pack(t)
# x = pickle.dumps(t)
len(x)

3

In [11]:
x

b'\x80\x03chumpack.basic_containers\ntdict\nq\x00)Rq\x01(X\x01\x00\x00\x00aq\x02K\x01X\x01\x00\x00\x00xq\x03h\x01X\x01\x00\x00\x00lq\x04]q\x05(chumpack.basic_containers\ntlist\nq\x06)\x81q\x07}q\x08(X\x05\x00\x00\x00_dataq\t]q\nX\x07\x00\x00\x00_shadowq\x0bNubchumpack.basic_containers\ntset\nq\x0c]q\r\x85q\x0eRq\x0f}q\x10(h\tccollections\nOrderedDict\nq\x11)Rq\x12h\x0bNubeKdX\x03\x00\x00\x00100q\x13NG?\xf3333333X\x01\x00\x00\x00mq\x14NX\x01\x00\x00\x00bq\x15C\x18\xe0\xe0\xc2\xc8\x9fk\xe5?j\x1e\xeeJ\xce\r\xf5?\x8b\xa9e\x84\xebx\xe1?q\x16C\x01\xaaq\x17X\x07\x00\x00\x00runningq\x18u}q\x19(h\th\x11)Rq\x1a(h\x02K\x01h\x03h\x01h\x04h\x05Kdh\x13NG?\xf3333333h\x14Nh\x15h\x16h\x17h\x18uh\x0bNub.'

In [6]:
# l = unpack(x)
l = scr.secure_unpack(x)
l

········


t{'a':1, 'x':{...}, 'l':[t[], t{}], 100:'100', None:1.2, 'm':None, 'b':b'\xe0\xe0\xc2\xc8\x9fk\xe5?j\x1e\xeeJ\xce\r\xf5?\x8b\xa9e\x84\xebx\xe1?', b'\xaa':'running'}

In [4]:
from cryptography.fernet import Fernet

In [5]:
f = Fernet('test')
f

ValueError: Fernet key must be 32 url-safe base64-encoded bytes.

In [6]:
key = Fernet.generate_key()
key

b'oghzOTH5VPfD0kclgHrkDh01AKHBF3MbBTtYXMicKd4='

In [3]:
import base64
import os
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

In [16]:
x = tdict()
x.a = 1
x.x = x
x.l = [tlist(), tset()]
x[100] = '100'
x[None] = 1.2
x.m = None
x.b = np.random.randn(3).tobytes()
x[b'\xaa'] = 'running'
x

t{'a':1, 'x':{...}, 'l':[t[], t{}], 100:'100', None:1.2, 'm':None, 'b':b"\xa1\xb5=\x92\xf2\t\xda\xbfX\xda(\x88',\xc4?#9\xd9<c\xa9\xd7\xbf", b'\xaa':'running'}

In [17]:
msg = json.dumps(pack(x)).encode('latin1')
msg

b'{"table": {"<>139994207180184": {"_type": "humpack.basic_containers.tdict", "_data": {"_pairs": {"a": 1, "x": "<>139994207180184", "l": "<>139994208494536", "<>94495694774400": "100", "<>94495694635248": 1.2, "m": null, "b": "<>139994207416944", "<>139994206928648": "running"}, "_order": ["a", "x", "l", "<>94495694774400", "<>94495694635248", "m", "b", "<>139994206928648"]}}, "<>139994208494536": {"_data": ["<>139994207409080", "<>139994206921008"], "_type": "list"}, "<>139994207409080": {"_type": "humpack.basic_containers.tlist", "_data": {"_entries": []}}, "<>139994206921008": {"_type": "humpack.basic_containers.tset", "_data": {"_elements": []}}, "<>94495694774400": {"_type": "int", "_data": 100}, "<>94495694635248": {"_type": "NoneType", "_data": null}, "<>139994207416944": {"_data": "\\u00a1\\u00b5=\\u0092\\u00f2\\t\\u00da\\u00bfX\\u00da(\\u0088\',\\u00c4?#9\\u00d9<c\\u00a9\\u00d7\\u00bf", "_type": "bytes"}, "<>139994206928648": {"_data": "\\u00aa", "_type": "bytes"}}, "meta": {

In [18]:
password = b'password'

kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
    backend=default_backend(),
)
key = base64.urlsafe_b64encode(kdf.derive(password))
f = Fernet(key)
token = f.encrypt(msg)
print(len(msg), len(token))
lmsg = f.decrypt(token)
lobj = unpack(json.loads(lmsg.decode('latin1')))
lobj

1013 1444


t{'a':1, 'x':{...}, 'l':[t[], t{}], 100:'100', None:1.2, 'm':None, 'b':b"\xa1\xb5=\x92\xf2\t\xda\xbfX\xda(\x88',\xc4?#9\xd9<c\xa9\xd7\xbf", b'\xaa':'running'}

In [23]:
salt = b'6FwLrxJb5mTPVwthumpackMASTERsalt'
len(salt)

32

In [25]:
f = Fernet(base64.urlsafe_b64encode(salt))

In [27]:
from crypt import crypt

In [29]:
crypt('test', salt.decode())

'6FYyxzQ.VE6WY'

In [3]:
id(t)

140002305622360

In [4]:
x = tdict()
x.a = 1
x.x = x
x.l = [tlist(), tset()]
x[100] = '100'
x[None] = 1.2
x.m = None
x.b = np.random.randn(3).tobytes()
x[b'\xaa'] = 'running'
x

t{'a':1, 'x':{...}, 'l':[t[], t{}], 100:'100', None:1.2, 'm':None, 'b':b'\xb8\x89\x89\x8fU\xda\xce\xbf\xcb\xfc\x9f\xf9\xe6S\xeb?\xd2A\xf72\xf2\x98\x00@', b'\xaa':'running'}

In [5]:
d = pack(x)
d

{'table': {'<>140002305124960': {'_type': 'humpack.basic_containers.tdict',
   '_data': {'_pairs': {'a': 1,
     'x': '<>140002305124960',
     'l': '<>140003119278856',
     '<>94722652590208': '100',
     '<>94722652451056': 1.2,
     'm': None,
     'b': '<>140002646422384',
     '<>140002305235664': 'running'},
    '_order': ['a',
     'x',
     'l',
     '<>94722652590208',
     '<>94722652451056',
     'm',
     'b',
     '<>140002305235664']}},
  '<>140003119278856': {'_data': ['<>140002305078280', '<>140002982137312'],
   '_type': 'list'},
  '<>140002305078280': {'_type': 'humpack.basic_containers.tlist',
   '_data': {'_entries': []}},
  '<>140002982137312': {'_type': 'humpack.basic_containers.tset',
   '_data': {'_elements': []}},
  '<>94722652590208': {'_type': 'int', '_data': 100},
  '<>94722652451056': {'_type': 'NoneType', '_data': None},
  '<>140002646422384': {'_data': '¸\x89\x89\x8fUÚÎ¿Ëü\x9fùæSë?ÒA÷2ò\x98\x00@',
   '_type': 'bytes'},
  '<>140002305235664': {'_data': 'ª

In [6]:
j = json.dumps(d)
j

'{"table": {"<>140002305124960": {"_type": "humpack.basic_containers.tdict", "_data": {"_pairs": {"a": 1, "x": "<>140002305124960", "l": "<>140003119278856", "<>94722652590208": "100", "<>94722652451056": 1.2, "m": null, "b": "<>140002646422384", "<>140002305235664": "running"}, "_order": ["a", "x", "l", "<>94722652590208", "<>94722652451056", "m", "b", "<>140002305235664"]}}, "<>140003119278856": {"_data": ["<>140002305078280", "<>140002982137312"], "_type": "list"}, "<>140002305078280": {"_type": "humpack.basic_containers.tlist", "_data": {"_entries": []}}, "<>140002982137312": {"_type": "humpack.basic_containers.tset", "_data": {"_elements": []}}, "<>94722652590208": {"_type": "int", "_data": 100}, "<>94722652451056": {"_type": "NoneType", "_data": null}, "<>140002646422384": {"_data": "\\u00b8\\u0089\\u0089\\u008fU\\u00da\\u00ce\\u00bf\\u00cb\\u00fc\\u009f\\u00f9\\u00e6S\\u00eb?\\u00d2A\\u00f72\\u00f2\\u0098\\u0000@", "_type": "bytes"}, "<>140002305235664": {"_data": "\\u00aa", "_t

In [7]:
uj = json.loads(j)
o = unpack(d)
o

t{'a':1, 'x':{...}, 'l':[t[], t{}], 100:'100', None:1.2, 'm':None, 'b':b'\xb8\x89\x89\x8fU\xda\xce\xbf\xcb\xfc\x9f\xf9\xe6S\xeb?\xd2A\xf72\xf2\x98\x00@', b'\xaa':'running'}

In [8]:
(np.frombuffer(x.b) - np.frombuffer(o.b)).sum() == 0

True

In [13]:
from crypt import crypt, mksalt
import getpass
from hmac import compare_digest

In [41]:
_master_salt = '$6$wGLrx8b5FmTPbhumpack-master-salt'

In [42]:
ps = 'test'
h = crypt(ps, _master_salt)

In [43]:
h = crypt(ps, h)
h

'$6$wGLrx8b5FmTPbhum$O/c7o6QbLy/7CHm.zOmYQ56yMHUwWp1rgEenWYy/2HRfJbs/l/8p66Ux7tmY4/.CFRnyzwGVvS32fqIVbkmjt0'

In [44]:
compare_digest(h, crypt(ps, h))

True

In [26]:
z = 1.0003+3j + np.pi
x = (z/1e18)*10003e13
x

(0.414313522138587+0.30008999999999997j)

In [27]:
complex(str(x)) == x

True

In [2]:
humpack.__version__

'0.1'

In [3]:
P = (type(None), str, float, int, bool)
P

(NoneType, str, float, int, bool)

In [4]:
Union[P]

typing.Union[NoneType, str, float, int, bool]

In [5]:
humpack.PRIMITIVE

typing.Union[NoneType, str, int, float, bool]

In [6]:
Union[P]

typing.Union[NoneType, str, float, int, bool]